In [303]:
# (CPU, DataSize, Y=Time)
import pandas as pd

In [304]:
df = pd.read_csv("testing.csv", sep=",") #data from 22July2016 
# copy the 'dup' folder to ~/ProfilingData/.
# remove *Output* files 
# run "python extract.py dup/" to generate testing.csv

In [305]:
len(df)

32

In [306]:
df.head(n=1)

,ppn,mem,seqlen,trial,AvgRSS,AvgSharedTxt,AvgStack,AvgTotal,AvgUnsharedData,CommandTimed,...,PageSize,PercentCPU,SignalsDeliv,SocketMsgRecv,SocketMsgSent,Swaps,SystemTime,UserTime,VolCtxtSwitch,WallTime
0,11,64G,10M,1,0,0,0,0,0,./NGS-ann-program/BIN/cd-hit-dup -i ./data/no-...,...,4096,99%,0,0,0,0,3.5,42.32,1072,0:45.97


In [307]:
#find which columns are completely empty
a=len(df[df['AvgRSS'] > 0])
b=len(df[df['AvgSharedTxt'] > 0])
c=len(df[df['AvgStack'] > 0])
d=len(df[df['AvgTotal'] > 0])
e=len(df[df['AvgUnsharedData'] > 0])

print(a,b,c,d,e)

0 0 0 0 0


In [308]:
header = list(df.columns)

In [309]:
i = 1
a = 0
emptyColumns = []
for h in header:
    try:
        a=len(df[df[h] > 0])
        print(a,  h)
        if(a==0): #nothing is greater than zero for this column
            emptyColumns.append(h)
    except:
        print('......................'+ h +' not a scalar')
    i=i+1

32 ppn
......................mem not a scalar
......................seqlen not a scalar
32 trial
0 AvgRSS
0 AvgSharedTxt
0 AvgStack
0 AvgTotal
0 AvgUnsharedData
......................CommandTimed not a scalar
0 ExitStatus
32 FileSysIn
32 FileSysOut
32 InvolCtxtSwitch
32 MajorFaults
32 MaxRSS
32 MinorFaults
32 PageSize
......................PercentCPU not a scalar
0 SignalsDeliv
0 SocketMsgRecv
0 SocketMsgSent
0 Swaps
32 SystemTime
32 UserTime
32 VolCtxtSwitch
......................WallTime not a scalar


In [310]:
emptyColumns.append('trial')
emptyColumns.append('PageSize')
emptyColumns.append('CommandTimed')
emptyColumns.append('WallTime')

In [311]:
for c in emptyColumns:
    del df[c]

In [312]:
df.head(1)

,ppn,mem,seqlen,FileSysIn,FileSysOut,InvolCtxtSwitch,MajorFaults,MaxRSS,MinorFaults,PercentCPU,SystemTime,UserTime,VolCtxtSwitch
0,11,64G,10M,3234924,2179426,212,3,6544812,1388550,99%,3.5,42.32,1072


In [313]:
ft = df.copy(deep=True)

In [314]:
delthis = list(set(ft.columns.values) - set(['ppn','seqlen','SystemTime','UserTime']))
delthis

['MinorFaults',
 'PercentCPU',
 'VolCtxtSwitch',
 'FileSysOut',
 'MaxRSS',
 'InvolCtxtSwitch',
 'mem',
 'FileSysIn',
 'MajorFaults']

In [315]:
for c in delthis:
    del ft[c]

In [316]:
#DROP the lines which are NULL
#df = df[df.CommandTimed.notnull()]
len(df)

32

In [317]:
whatisthesize = {'5M': .54, '10M': 1.2,'15M': 1.7,'20M': 2.3,'25M': 2.9,'30M': 3.4,'35M': 4.0,'40M': 4.5}

In [318]:
ft['sizeGB'] = ft['seqlen'].map(whatisthesize)

In [319]:
del ft['seqlen']
ft.head(2)

,ppn,SystemTime,UserTime,sizeGB
0,11,3.50,42.32,1.2
1,11,5.43,62.54,1.7


In [320]:
ft['Y_time']=ft['SystemTime']+ft['UserTime']
del ft['SystemTime']
del ft['UserTime']
ft.head()

,ppn,sizeGB,Y_time
0,11,1.2,45.82
1,11,1.7,67.97
2,11,2.3,98.47
3,11,2.9,120.44
4,11,3.4,141.66


In [321]:
from sklearn.ensemble.forest import RandomForestRegressor
import numpy as  np

In [322]:
model = RandomForestRegressor()

In [323]:
np.random.seed()
ft['is_train']=np.random.uniform(0,1,len(ft)) <= .70
train, test = ft[ft['is_train']==True], ft[ft['is_train']==False]
del ft['is_train']
len(train)*1.0 / (len(test)+len(train))

0.625

In [324]:
ft.columns[0:2]

Index(['ppn', 'sizeGB'], dtype='object')

In [325]:
features = ft.columns[0:2]
target = ft.columns[2]
model.fit(train[features], train[target].values)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [326]:
Y_prediction = model.predict(test[features])
Y_test = test[target].as_matrix()

In [327]:
model.score(test[features], test[target]) #which score is this ?

0.95224672297544144

In [328]:
from sklearn import cross_validation
scores = cross_validation.cross_val_score(model, ft[features], ft[target], cv=10,scoring='mean_squared_error')

In [329]:
scores

array([ -3.1050665 , -13.2615825 ,  -1.427065  ,  -2.949084  ,
       -31.856599  ,  -2.43790167, -36.831207  ,  -1.795295  ,
        -6.370602  ,  -8.86424167])

In [330]:
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: -10.89 (+/- 24.58)


In [331]:
ft[target].mean(), ft[target].std() * 2

(108.51031250000001, 118.18468799728724)

In [332]:
scores = cross_validation.cross_val_score(model, ft[features], ft[target], cv=10,scoring='r2')
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.98 (+/- 0.04)
